<a href="https://colab.research.google.com/github/mateusplima/imersao-alura-google/blob/main/Projeto_Alura_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [58]:
# Instalar Framework de agentes do Google #
!pip install -q google-adk

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import io

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [62]:
##############################################
# --- Agente 1: Extração de Perfil (Simplificado) --- #
##############################################

def obter_perfil_do_usuario_simplificado():
    """
    Função para coletar informações básicas do usuário para a busca de vagas.
    """
    print("Bem-vindo ao Agente de Extração de Perfil!")
    print("Por favor, responda às seguintes perguntas:")

    area_atuacao = input("Área de atuação pretendida: ")
    local_trabalho = input("Local de trabalho de preferência: ")
    salario_base = input("Valor base do salário desejado (opcional): ")
    tempo_experiencia = input("Tempo de experiência na função (ex: 2 anos, 5 anos): ")
    aceita_remoto = input("Aceita trabalho remoto? (sim/não)").lower()
    disposto_mudar = input("Disposto a trabalhar em cidades próximas? (sim/não): ").lower()


    perfil_texto = f"""
    Área de atuação pretendida: {area_atuacao}
    Local de trabalho de preferência: {local_trabalho}
    Valor base do salário desejado: {salario_base}
    Tempo de experiência na função: {tempo_experiencia}
    Aceita trabalho remoto? {aceita_remoto}
    Disposto a trabalhar em cidades próximas? {disposto_mudar}
    """

    return perfil_texto



# Criação do Agente de Extração de Perfil (Simplificado)
agente_extrator = Agent(
    name="agente_extrator_perfil",
    model=MODEL_ID,
    instruction="""
    Você é um especialista em análise de informações de candidatos para busca de vagas de emprego.
    Sua tarefa é organizar as informações fornecidas pelo usuário em um formato de texto claro e conciso.

    As informações fornecidas incluem:
    - Área de atuação pretendida: [Área de atuação]
    - Local de trabalho de preferência: [Local de trabalho]
    - Valor base do salário desejado: [Salário]
    - Tempo de experiência na função: [Tempo de experiência]

    Organize essas informações em um texto que destaque os principais critérios de busca do candidato.
    """,
    description="Agente que organiza as informações básicas do candidato"
)

# Função para obter o perfil do usuário e passá-lo pelo agente
def obter_perfil_processado():
    perfil_texto = obter_perfil_do_usuario_simplificado()
    if perfil_texto:
        resultado_extracao = call_agent(agente_extrator, perfil_texto)
        return resultado_extracao
    else:
        return None

In [72]:
##############################################
# --- Agente 2: Busca de Vagas --- #
##############################################

data_atual = datetime.now().strftime("%Y-%m-%d")
data_limite = (datetime.now() - timedelta(days=180)).strftime("%Y-%m-%d")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    """
    Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final.
    """
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response.strip() # Remova espaços em branco extras



def agente_buscador_vagas(perfil_candidato: str):
    """
    Agente que utiliza a busca do Google para encontrar vagas de emprego
    com base no perfil do candidato. Retorna a resposta do agente.
    """
    data_atual = datetime.now().strftime("%Y-%m-%d")
    data_limite = (datetime.now() - timedelta(days=90)).strftime("%Y-%m-%d")

    buscador_vagas = Agent(
        name="agente_buscador_vagas",
        model=MODEL_ID,
        instruction=f"""

        Sua tarefa é usar a ferramenta de busca do Google para encontrar vagas de emprego relevantes para o seguinte perfil de candidato:
        Se o candidato aceitar remoto, inclua termos como 'remoto', 'home office' nas buscas. Se estiver disposto a mudar, considere
        buscar em regiões metropolitanas ou cidades vizinhas mencionadas

        {perfil_candidato}

        Crie pelo menos 3 consultas de busca diferentes e eficazes, combinando palavras-chave da área de atuação, local de trabalho e tempo de experiência. Por exemplo, para um 'Desenvolvedor Python em São Paulo com 3 anos de experiência', algumas consultas poderiam ser:
        - "vaga desenvolvedor Python São Paulo"
        - "oportunidade Python pleno São Paulo"
        - "desenvolvedor Python 3 anos experiência remoto ou São Paulo"
        Inclua também variações considerando sinônimos e termos relacionados.

        Execute cada uma dessas consultas de busca e compile os resultados. As vagas encontradas devem ter sido abertas e estar disponíveis nos últimos 180 dias (entre {data_limite} e {data_atual}).

        """,
        description="Agente que busca vagas de emprego no Google com base no perfil do candidato",
        tools=[google_search]
    )

    entrada_do_agente_buscador_vagas = f"Perfil do Candidato:\n{perfil_candidato}\n\nData Atual (para referência de tempo): {data_atual}"

    # Use a função auxiliar call_agent para executar o agente
    resultado_busca_text = call_agent(buscador_vagas, entrada_do_agente_buscador_vagas)

     # Tenta carregar a resposta como JSON

    return resultado_busca_text

In [79]:
##############################################
# --- Agente 3: Filtragem e Compatibilidade de Vagas --- #
##############################################

def agente_compatibilidade_vagas(perfil_candidato: str, resultados_busca_vagas: str) -> str:
    """
    Agente que compara o perfil do candidato com as descrições das vagas
    e retorna as 10 vagas mais compatíveis com uma pontuação.
    Usa a função auxiliar call_agent para interagir com o agente.
    """
    agente_compatibilidade = Agent(
        name="agente_compatibilidade",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em recrutamento e análise de compatibilidade de candidatos com vagas de emprego.
        Sua tarefa é analisar o perfil do candidato fornecido e compará-lo com a lista de vagas encontradas.

        Perfil do Candidato:
        {perfil_candidato}

        Lista de Vagas Encontradas:
        {resultados_busca_vagas}

        Para cada vaga na lista, avalie o quão bem o perfil do candidato se encaixa nos requisitos, habilidades e experiência descritos.
        Atribua uma pontuação de compatibilidade para cada vaga em uma escala de 0 a 100, onde 100 indica uma correspondência perfeita.
        Justifique brevemente a sua pontuação para cada vaga, mencionando os principais pontos de compatibilidade ou incompatibilidade.

        Após avaliar todas as vagas, filtre a lista para incluir apenas as vagas com uma pontuação de compatibilidade **maior que 50**.
        Crie uma lista com no máximo 10 dessas vagas mais indicadas para este perfil, ordenadas pela pontuação de compatibilidade em ordem decrescente.
        Se não houver vagas com compatibilidade maior que 40, retorne uma mensagem indicando que nenhuma vaga compatível foi encontrada dentro desse critério.
    """,
    description="Agente que avalia a compatibilidade entre o perfil do candidato e as vagas encontradas (retornando apenas vagas com compatibilidade > 40)"
)

    entrada_do_agente_compatibilidade = f"Perfil do Candidato:\n{perfil_candidato}\n\nResultados da Busca de Vagas:\n{resultados_busca_vagas}"

    # Use a função auxiliar call_agent para executar o agente
    resultado_compatibilidade_text = call_agent(agente_compatibilidade, entrada_do_agente_compatibilidade)

    # A função call_agent já retorna o texto final da resposta
    return resultado_compatibilidade_text

In [77]:
##############################################
# --- Agente 4: Apresentação de Vagas --- #
##############################################

def agente_apresentacao_vagas(vagas_recomendadas: str):
    """
    Agente que formata e exibe as vagas mais adequadas para o usuário.
    """
    agente_apresentacao = Agent(
        name="agente_apresentacao",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em apresentação de resultados de busca de vagas de emprego.
        Sua tarefa é formatar a lista de vagas recomendadas para o usuário de forma clara, concisa e amigável.

        Lista de Vagas Recomendadas:
        {vagas_recomendadas}

        Organize as informações de cada vaga de forma legível, destacando os pontos mais importantes para o usuário.
        Apresente as vagas em uma lista numerada, ordenada pela pontuação de compatibilidade (da maior para a menor).
        Use uma linguagem clara e profissional.

        Para cada vaga, inclua as seguintes informações, formatadas em Markdown:

        **Título da Vaga:** [Título da Vaga]
        **Empresa:** [Nome da Empresa]
        **Localização:** [Localização]
        **Pontuação de Compatibilidade:** [Pontuação]/100
        **Por que essa vaga é interessante:** Mencione **um ou dois** principais motivos pelos quais essa vaga se alinha ao perfil do candidato.
        **Link:** [Link]

        ---
        """,
        description="Agente que formata e apresenta as vagas de emprego recomendadas ao usuário"
    )

    entrada_do_agente_apresentacao = f"Lista de Vagas Recomendadas:\n{vagas_recomendadas}"

    # Use a função auxiliar call_agent para executar o agente
    resultado_apresentacao = call_agent(agente_apresentacao, entrada_do_agente_apresentacao)

    # A função call_agent já retorna o texto final da resposta
    return resultado_apresentacao





In [78]:
##############################################
# --- Orquestração dos Agentes --- #
##############################################

def iniciar_busca_vagas_orquestrada():
    """
    Função para iniciar a busca de vagas de emprego, orquestrando os agentes.
    """
    perfil_usuario = obter_perfil_processado()

    if perfil_usuario:
        print("\n--- Perfil do Usuário Processado ---")
        to_markdown(perfil_usuario)

        vagas_encontradas = agente_buscador_vagas(perfil_usuario)
        if vagas_encontradas:
            print("\n--- Vagas Encontradas ---")
            to_markdown(vagas_encontradas)

            vagas_recomendadas = agente_compatibilidade_vagas(perfil_usuario, vagas_encontradas)
            if vagas_recomendadas:
                to_markdown(vagas_recomendadas)

                resultado_final = agente_apresentacao_vagas(vagas_recomendadas)
                print("\n--- Vagas Mais Indicadas para Você ---")
                to_markdown(resultado_final)
                print(resultado_final)
            else:
                print("\nNenhuma vaga altamente compatível foi encontrada.")
        else:
            print("\nNenhuma vaga foi encontrada com base no seu perfil.")
    else:
        print("\nNão foi possível obter as informações do seu perfil.")


# Iniciar o processo de busca de vagas orquestrado
if __name__ == "__main__":
    iniciar_busca_vagas_orquestrada()


Bem-vindo ao Agente de Extração de Perfil!
Por favor, responda às seguintes perguntas:
Área de atuação pretendida: costureira
Local de trabalho de preferência: fortaleza
Valor base do salário desejado (opcional): 1412
Tempo de experiência na função (ex: 2 anos, 5 anos): 4 anos
Aceita trabalho remoto? (sim/não)sim
Disposto a trabalhar em cidades próximas? (sim/não): sim

--- Perfil do Usuário Processado ---

--- Vagas Encontradas ---

--- Vagas Mais Indicadas para Você ---
Olá! Aqui estão as vagas de emprego mais indicadas para você, com base no seu perfil e nas buscas realizadas:

1.  **Título da Vaga:** Vagas de Costureira no Indeed (se encontradas após filtragem)
    **Empresa:** Indeed
    **Localização:** Fortaleza
    **Pontuação de Compatibilidade:** 75/100
    **Por que essa vaga é interessante:** Sua experiência como costureira e sua preferência por trabalhar em Fortaleza tornam essa opção altamente relevante.
    **Link:** [Link para o Indeed](https://br.indeed.com/)

    ---
